In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnableLambda
from langchain_community.chat_models import ChatOllama

# Create a ChatOpenAI model
model = ChatOllama(model="llama3:8b")

In [13]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an expert product reviewer."),
        ("human", "List the main features of the product {product_name}."),
    ]
)


In [14]:
# Define pros analysis step
def analyze_pros(features):
    pros_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the pros of these features.",
            ),
        ]
    )
    return pros_template.format_prompt(features=features)


# Define cons analysis step
def analyze_cons(features):
    cons_template = ChatPromptTemplate.from_messages(
        [
            ("system", "You are an expert product reviewer."),
            (
                "human",
                "Given these features: {features}, list the cons of these features.",
            ),
        ]
    )
    return cons_template.format_prompt(features=features)


In [15]:
# Combine pros and cons into a final review
def combine_pros_cons(pros, cons):
    return f"Pros:\n{pros}\n\nCons:\n{cons}"

In [16]:
# Simplify branches with LCEL
pros_branch_chain = (
    RunnableLambda(lambda x: analyze_pros(x)) | model | StrOutputParser()
)

cons_branch_chain = (
    RunnableLambda(lambda x: analyze_cons(x)) | model | StrOutputParser()
)

In [17]:
# Create the combined chain using LangChain Expression Language (LCEL)
chain = (
    prompt_template
    | model
    | StrOutputParser()
    | RunnableParallel(branches={"pros": pros_branch_chain, "cons": cons_branch_chain})
    | RunnableLambda(lambda x: combine_pros_cons(x["branches"]["pros"], x["branches"]["cons"]))
)

In [18]:
# Run the chain
result = chain.invoke({"product_name": "MacBook Pro"})

In [19]:
# Output
print(result)

Pros:
Based on the features listed for the MacBook Pro, here are some potential pros:

**Display:**

* High-resolution display for crisp text and vibrant colors
* Ideal for creative professionals who require precise color representation (e.g., graphic designers, photographers)

**Processors:**

* Exceptional processing power for demanding tasks like video editing, software development, etc.
* Enables fast performance for multitasking, making it suitable for professionals who need to work on multiple projects simultaneously

**Memory and Storage:**

* Ample memory and storage capacity for running multiple apps and files simultaneously without slowing down
* Ideal for professionals who require large file sizes (e.g., video editors, software developers)

**Graphics:**

* Excellent performance for graphics-intensive tasks like gaming, video editing, etc.
* Enables smooth rendering of high-resolution images and videos

**Battery Life:**

* Up to 10 hours of battery life, making it suitable 